<a href="https://colab.research.google.com/github/jonlwowski012/StockSellRulesChecker/blob/master/StockScreener/Screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
! git clone https://github.com/jonlwowski012/StockSellRulesChecker.git
! pip install -r StockSellRulesChecker/StockScreener/requirements.txt
! pip install -U git+https://github.com/mariostoev/finviz

fatal: destination path 'StockSellRulesChecker' already exists and is not an empty directory.
  Cloning https://github.com/mariostoev/finviz to /tmp/pip-req-build-ss4nk0d0
  Running command git clone -q https://github.com/mariostoev/finviz /tmp/pip-req-build-ss4nk0d0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for finviz: filename=finviz-1.3.4-cp37-none-any.whl size=16979 sha256=caa1a4e8ee74e46e6be70cbbaac7ac011231b18fe1ffa048f6314cdd21387ace
  Stored in directory: /tmp/pip-ephem-wheel-cache-3lg6eixv/wheels/98/cc/6a/68893eb662f0f196f8e153e9ce4424a8ae9e1f022fd64e4d6d
Successfully built finviz
  Found existing installation: finviz 1.3.4
    Uninstalling finviz-1.3.4:
      Successfully uninstalled finviz-1.3.4


In [1]:
from finviz.screener import Screener
import nest_asyncio
import finviz
import pandas_datareader as pdr
import yfinance as yf
yf.pdr_override() # <== that's all it takes :-)

import datetime
import stockquotes
import pandas as pd
import tqdm
nest_asyncio.apply()

import time

In [3]:
eps_5year_over5pct = True
eps_qoq_over5pct = True
eps_yoy_over5pct = True
sales_5years_over5pct = True
sales_qoq_over5pct = True

filters = []
if eps_5year_over5pct:
  filters.append('fa_eps5years_o5')
if eps_qoq_over5pct:
  filters.append('fa_epsqoq_o5')
if eps_yoy_over5pct:
  filters.append('fa_epsyoy_o5')
if sales_5years_over5pct:
  filters.append('fa_sales5years_o5')
if sales_qoq_over5pct:
  filters.append('fa_salesqoq_o5')

stock_list = Screener(filters=filters, table='Performance', order='price')  

In [4]:
print(len(stock_list.data))

356


In [5]:
def moving_average(yahoo_df, days):
    df = yahoo_df.reset_index()
    curr_date = df['Date'].max()
    end_date = curr_date - datetime.timedelta(days=days)
    
    df_days = df[df['Date'] >= end_date]
    return float(df_days['Close'].mean())

def week52_low_high(yahoo_df):
    df = yahoo_df.reset_index()
    curr_date = df['Date'].max()
    end_date = curr_date - datetime.timedelta(days=365)
    
    df_days = df[df['Date'] >= end_date]
    return float(df_days['Close'].max()), float(df_days['Close'].min())


screened_stocks = {}
for stock in tqdm.tqdm(stock_list, total=len(stock_list.data)):
  print(stock['Ticker'])
  if stock["Ticker"] == "":
    continue
  screened_stocks[stock['Ticker']] = {}
  finviz_stats = finviz.get_stock(stock['Ticker'])
  try:
      yahoo_df = pdr.get_data_yahoo(stock['Ticker'], interval = "d")
  except:
    continue

  SMA200_value = moving_average(yahoo_df, days=200)
  SMA150_value = moving_average(yahoo_df, days=150)
  SMA50_value = moving_average(yahoo_df, days=50)
  SMA200_percent = float(finviz_stats['SMA200'].replace("%",""))
  SMA50_percent = float(finviz_stats['SMA50'].replace("%",""))

  try:
    EPS_ttm = float(finviz_stats['EPS (ttm)'])
  except:
    EPS_ttm = 0

  try:
    EPS_QoQ_percent = float(finviz_stats['EPS Q/Q'].replace("%",""))
  except:
    EPS_QoQ_percent = 0

  try:
    Sales_QoQ_percent = float(finviz_stats['Sales Q/Q'].replace("%",""))
  except:
    Sales_QoQ_percent = 0

  try:
    inst_own = float(finviz_stats['Inst Own'].replace("%",""))/100
  except:
    inst_own = 0
  
  shares_outstanding = finviz_stats['Shs Outstand']
  if 'M' in shares_outstanding:
    shares_outstanding = float(shares_outstanding.replace("M", ""))*1e6
  elif 'B' in shares_outstanding:
    shares_outstanding = float(shares_outstanding.replace("B", ""))*1e9
  elif 'T' in shares_outstanding:
    shares_outstanding = float(shares_outstanding.replace("T", ""))*1e12
  else:
    shares_outstanding = 0

  prev_close = float(finviz_stats['Prev Close'].replace("$",""))
  week52_high, week52_low = week52_low_high(yahoo_df)
  
  screened_stocks[stock['Ticker']]['SMA200_value'] = SMA200_value
  screened_stocks[stock['Ticker']]['SMA150_value'] = SMA150_value
  screened_stocks[stock['Ticker']]['SMA50_value'] = SMA50_value
  screened_stocks[stock['Ticker']]['SMA200_percent'] = SMA200_percent
  screened_stocks[stock['Ticker']]['SMA50_percent'] = SMA50_percent
  screened_stocks[stock['Ticker']]['EPS_QoQ_percent'] = EPS_QoQ_percent
  screened_stocks[stock['Ticker']]['EPS_ttm'] = EPS_ttm
  screened_stocks[stock['Ticker']]['Sales_QoQ_percent'] = Sales_QoQ_percent
  screened_stocks[stock['Ticker']]['prev_close'] = prev_close
  screened_stocks[stock['Ticker']]['week52_high'] = week52_high
  screened_stocks[stock['Ticker']]['week52_low'] = week52_low
  screened_stocks[stock['Ticker']]['Inst. Ownership'] = inst_own
  screened_stocks[stock['Ticker']]['Shares Outstanding'] = shares_outstanding
  
  # EPS TTM > $1 Rule
  if EPS_ttm >= 1.0:
    eps_ttm_rule = True
  else:
    eps_ttm_rule = False

  # EPS QoQ Yearly > 18% rule
  if EPS_QoQ_percent >= 0.18:
    eps_QoQ_yearly_rule = True
  else:
    eps_QoQ_yearly_rule = False
  screened_stocks[stock['Ticker']]['eps_QoQ_yearly_rule'] = eps_QoQ_yearly_rule

  # Sales QoQ Yearly > 25% rule
  if Sales_QoQ_percent >= 0.25:
    sales_QoQ_yearly_rule = True
  else:
    sales_QoQ_yearly_rule = False
  screened_stocks[stock['Ticker']]['sales_QoQ_yearly_rule'] = sales_QoQ_yearly_rule

  # Shares Outstanding <= 25 mil
  if shares_outstanding <= 25e6:
    shares_outstanding_rule = True
  else:
    shares_outstanding_rule = False
  screened_stocks[stock['Ticker']]['shares_outstanding_rule'] = shares_outstanding_rule

  # Institutional Ownership < 35%
  if 0.05 <= inst_own <= 0.35:
    inst_ownership_rule = True
  else:
    inst_ownership_rule = False
  screened_stocks[stock['Ticker']]['inst_ownership_rule'] = inst_ownership_rule

  # Positive 200d MA positive
  if SMA200_percent > 0:
      SMA200_slope_rule = True
  else:
      SMA200_slope_rule = False
  screened_stocks[stock['Ticker']]['SMA200_slope_rule'] = SMA200_slope_rule
      
  # 150d MA greater than 200d MA
  if SMA150_value > SMA200_value:
      SMA150_greater_SMA200_rule = True
  else:
      SMA150_greater_SMA200_rule = False
  screened_stocks[stock['Ticker']]['SMA150_greater_SMA200_rule'] = SMA150_greater_SMA200_rule
      
  # 50d MA greater than 150d MA
  if SMA50_value > SMA150_value:
      SMA50_greater_SMA150_rule = True
  else:
      SMA50_greater_SMA150_rule = False
  screened_stocks[stock['Ticker']]['SMA50_greater_SMA150_rule'] = SMA50_greater_SMA150_rule
      
  # Close above 50d MA
  if prev_close > SMA50_value:
      close_greater_SMA50_rule = True
  else:
      close_greater_SMA50_rule = False
  screened_stocks[stock['Ticker']]['close_greater_SMA50_rule'] = close_greater_SMA50_rule
      
  # 52 week high low span rule
  if 0.75*week52_high > 1.25*week52_low:
      week52_span_rule = True
  else:
      week52_span_rule = False
  screened_stocks[stock['Ticker']]['week52_span_rule'] = week52_span_rule
  
  # Close above 52 week high - 25%
  if prev_close > 0.75*week52_high:
      close_above_52weekhigh_rule = True
  else:
      close_above_52weekhigh_rule = False
  screened_stocks[stock['Ticker']]['close_above_52weekhigh_rule'] = close_above_52weekhigh_rule

output_list = []
for stock in screened_stocks.keys():
    cols = ["Ticker"] + list(screened_stocks[stock].keys())
    temp_list = []
    temp_list.append(stock)
    for rule in screened_stocks[stock].keys():
        temp_list.append(screened_stocks[stock][rule])
    output_list.append(temp_list)
        
df_out = pd.DataFrame(output_list,columns=cols)

    


  0%|          | 0/356 [00:00<?, ?it/s]

ISR


  0%|          | 1/356 [00:00<05:03,  1.17it/s]

NBRV


  1%|          | 2/356 [00:01<04:47,  1.23it/s]

...

 99%|█████████▉| 354/356 [09:52<00:03,  1.84s/it]

GOOGL


100%|█████████▉| 355/356 [09:54<00:01,  1.75s/it]

AMZN


100%|██████████| 356/356 [09:56<00:00,  1.67s/it]


In [6]:
rules = [col for col in df_out.columns if "rule" in col]
df_out['Passed Tests'] = (df_out[list(rules)]).sum(1)

In [7]:
df_out.sort_values('Passed Tests', ascending=False)

,Ticker,SMA200_value,SMA150_value,SMA50_value,SMA200_percent,SMA50_percent,EPS_QoQ_percent,EPS_ttm,Sales_QoQ_percent,prev_close,week52_high,week52_low,Inst. Ownership,Shares Outstanding,eps_QoQ_yearly_rule,sales_QoQ_yearly_rule,shares_outstanding_rule,inst_ownership_rule,SMA200_slope_rule,SMA150_greater_SMA200_rule,SMA50_greater_SMA150_rule,close_greater_SMA50_rule,week52_span_rule,close_above_52weekhigh_rule,Passed Tests
50,MYO,7.792333,8.928750,13.004167,116.46,17.45,92.4,-9.15,153.3,14.25,18.139999,3.000000,0.268,4.620000e+06,True,True,True,True,True,True,True,True,True,True,10.0
110,ATLC,19.844348,22.336442,28.946111,76.98,7.83,106.1,2.90,21.3,31.54,33.070000,5.350000,0.170,1.452000e+07,True,True,True,True,True,True,True,True,True,True,10.0
74,NODK,17.102681,17.381154,18.027500,17.78,10.92,29.5,1.85,18.6,19.68,19.910000,10.330000,0.243,2.142000e+07,True,True,True,True,True,True,True,True,True,True,10.0
72,CFBK,15.469203,16.687308,18.873889,38.05,5.24,435.7,4.25,245.5,19.40,20.250000,9.470000,0.323,6.270000e+06,True,True,True,True,True,True,True,True,True,True,10.0
68,OVLY,15.942971,16.869519,17.067778,24.94,12.30,15.8,1.68,8.3,18.60,20.250000,11.030000,0.222,8.130000e+06,True,True,True,True,True,True,True,True,True,True,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,AEM,72.255870,69.406346,63.059166,-15.84,-8.36,186.0,2.11,43.6,61.42,86.470001,36.360001,0.781,2.433000e+08,True,True,False,False,False,False,False,False,True,False,3.0
38,LFC,11.267101,11.134231,10.726944,-7.10,-3.29,85.9,1.40,26.3,10.66,13.430000,8.520000,0.011,5.650000e+09,True,True,False,False,False,False,False,False,False,True,3.0
261,QDEL,204.423769,203.668655,188.001111,-36.51,-31.04,1435.1,18.59,431.7,142.80,301.959991,80.690002,0.827,4.222000e+07,True,True,False,False,False,False,False,False,True,False,3.0
55,SSRM,18.380000,17.881538,15.806667,-19.53,-6.98,110.4,0.65,108.7,15.64,24.500000,10.770000,0.622,2.200100e+08,True,True,False,False,False,False,False,False,True,False,3.0


In [8]:
df_out[df_out["Passed Tests"] == 10]

,Ticker,SMA200_value,SMA150_value,SMA50_value,SMA200_percent,SMA50_percent,EPS_QoQ_percent,EPS_ttm,Sales_QoQ_percent,prev_close,week52_high,week52_low,Inst. Ownership,Shares Outstanding,eps_QoQ_yearly_rule,sales_QoQ_yearly_rule,shares_outstanding_rule,inst_ownership_rule,SMA200_slope_rule,SMA150_greater_SMA200_rule,SMA50_greater_SMA150_rule,close_greater_SMA50_rule,week52_span_rule,close_above_52weekhigh_rule,Passed Tests
10,MOSY,2.497609,2.801923,4.383611,81.89,5.22,76.0,-1.44,66.7,4.71,6.260000,0.840000,0.096,3550000.0,True,True,True,True,True,True,True,True,True,True,10.0
50,MYO,7.792333,8.928750,13.004167,116.46,17.45,92.4,-9.15,153.3,14.25,18.139999,3.000000,0.268,4620000.0,True,True,True,True,True,True,True,True,True,True,10.0
68,OVLY,15.942971,16.869519,17.067778,24.94,12.30,15.8,1.68,8.3,18.60,20.250000,11.030000,0.222,8130000.0,True,True,True,True,True,True,True,True,True,True,10.0
72,CFBK,15.469203,16.687308,18.873889,38.05,5.24,435.7,4.25,245.5,19.40,20.250000,9.470000,0.323,6270000.0,True,True,True,True,True,True,True,True,True,True,10.0
74,NODK,17.102681,17.381154,18.027500,17.78,10.92,29.5,1.85,18.6,19.68,19.910000,10.330000,0.243,21420000.0,True,True,True,True,True,True,True,True,True,True,10.0
84,ESCA,20.568696,21.164038,21.815000,18.68,2.20,98.5,1.82,59.1,22.91,24.049999,4.750000,0.307,14030000.0,True,True,True,True,True,True,True,True,True,True,10.0
103,MPB,22.221304,23.362019,24.894444,28.69,12.00,36.9,3.09,6.5,27.30,29.410000,15.900000,0.219,8410000.0,True,True,True,True,True,True,True,True,True,True,10.0
106,RBNC,18.707319,20.018269,22.908056,60.16,28.50,99.0,1.89,73.0,28.20,28.200001,9.740000,0.328,16620000.0,True,True,True,True,True,True,True,True,True,True,10.0
110,ATLC,19.844348,22.336442,28.946111,76.98,7.83,106.1,2.90,21.3,31.54,33.070000,5.350000,0.170,14520000.0,True,True,True,True,True,True,True,True,True,True,10.0
149,SAL,37.985580,39.163365,41.332222,16.95,10.01,204.6,4.18,197.3,44.75,50.880001,25.129999,0.238,2790000.0,True,True,True,True,True,True,True,True,True,True,10.0
